In [34]:
RAW_FILE_PATH = '../../data/raw/Transaction.csv'
OUTPUT_PATH = '../../data/'

In [35]:
%run '../../schema_definition/schema.py'

In [36]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [37]:
from pyspark.shell import spark
from pyspark.sql import functions as F

In [39]:
gold_df = spark.read.json(OUTPUT_PATH + "gold", gold_schema)
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT *
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
# fastest_response_df.printSchema()
# fastest_response_df.show(5, truncate=False)
import databricks.koalas as ks
koalas_df = ks.DataFrame(fastest_response_df)
print(koalas_df)

21/10/16 02:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


     AccountID  CODE ImplementedDate  ActiveIndicator AccountType Service    BU RequestDate AccountStatus  StatusCode   Amount Version  AgentID                     Fibre     LastUpdatedDate PropertyType  PostCode                                                           HashKey  Response
0        10046   105      2017-09-05                1         B2B     ELE  NUOS  2017-09-04          PAID           6     0.00    v1_0   307312    EA983_74004197612-N-11 2017-09-11 08:53:00         RESI      2581  19e635adb1867a76addf356de890d7497199057b6af7875ee14f802aa039c999        -1
1        10066   105      2017-09-05                1         B2B     ELE  NUOS  2017-09-04          PAID           5     0.00    v1_0   307312    EA983_78004104538-N-11 2017-09-11 09:25:00         RESI      2581  782974fe34aaaed7094ee399e9d592f17bc1b181da3f06c0b2f6483d324f73d3        -1
2        10035   105      2017-09-05                1         B2B     ELE  NUOS  2017-09-04          PAID           5    21.44    v1_